# Neighborhoods in Toronto Project
### import library

In [1]:
import pandas as pd
import numpy as np 
import urllib.request
from bs4 import BeautifulSoup

### Retrieve toronto postal code data

open url with urllib and getting transforming data with beautiful soup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [4]:
soup = BeautifulSoup(page,"html")

making a fonction for transforming td html content to a clean text

In [5]:
def normalize_text(text):
    return text.find(text=True).replace("\n","").strip()

1. define a dictionnary that will content all data
2. find data from html table
3. add data into the dictionnary depending on criteria

In [6]:
data = {'postal_code': [], 'Borough': [], 'Neighborhood': []}
table=soup.find('table', class_='wikitable sortable')
for row in table.findAll("tr"):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_code = normalize_text(cells[0]) 
        borough = normalize_text(cells[1])
        neigh  = normalize_text(cells[2])
        if borough.lower() != 'not assigned':
            if neigh.lower() == 'not assigned':
                neigh = borough
            if postal_code not in data["postal_code"]:
                data["postal_code"].append(postal_code)
                data["Borough"].append(borough)
                data["Neighborhood"].append(neigh)
            else:
                index = data["postal_code"].index(postal_code)
                data["Neighborhood"][index] = "{},{}".format(data["Neighborhood"][index],neigh)

AttributeError: 'NoneType' object has no attribute 'findAll'

transform data into dataframe and print the result

In [6]:
df = pd.DataFrame(data['postal_code'], columns=['Postal Code'])
df['Borough'] = data['Borough']
df['Neighboorhood'] = data['Neighborhood']
df.head()

,Postal Code,Borough,Neighboorhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df.shape

(103, 3)

In [24]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")

In [39]:
geo_df.shape

(103, 3)

In [34]:
df_ll = pd.merge(df,geo_df,on='Postal Code')

In [36]:
df_ll.head(10)

,Postal Code,Borough,Neighboorhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Use geopy ti get the latitude and longitude of toronto city 

In [58]:
from geopy.geocoders import Nominatim 
import re
import folium

In [54]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


getting only boroughs containing the name Toronto

In [56]:
toronto_data = df_ll[df_ll['Borough'].str.contains('Toronto',flags=re.IGNORECASE, regex=True)].reset_index(drop=True)
toronto_data.shape

(39, 5)

explore toronto neighboorhood

In [63]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label,categorie, in zip(toronto_data['Latitude'], toronto_data['Longitude'],
                                      toronto_data['Neighboorhood'],toronto_data):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto